In [1]:
import numpy as np
import pandas as pd

In [2]:
year = 2017
election_round = "first_round" # first_round, runoff

In [3]:
df = pd.read_excel(f"data/Chile/{year}_{election_round}.xlsx")
df.head()

,Región,Provincia,Circ.Senatorial,Distrito,Comuna,Circ.Electoral,Local,Mesa,Tipo mesa,Mesas Fusionadas,Electores,Nro.voto,Candidato,Votos TRICEL
0,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,1.0,CAROLINA GOIC BOROEVIC,3.0
1,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,2.0,JOSE ANTONIO KAST RIST,6.0
2,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,3.0,SEBASTIAN PIÑERA ECHENIQUE,96.0
3,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,4.0,ALEJANDRO GUILLIER ALVAREZ,19.0
4,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,5.0,BEATRIZ SANCHEZ MUÑOZ,12.0


In [4]:
# list(df)

In [5]:
if year == 2017:
    df["polling_id"] = df["Comuna"] + "-" + df["Circ.Electoral"] + "-" + df["Local"] + "-" + df["Mesa"].fillna(0).astype(int).astype(str)\
        + "-" + df["Tipo mesa"].fillna("NA") + "-" + df["Mesas Fusionadas"].astype(str)

elif year == 2021:
    df["polling_id"] = df["Comuna"] + "-" + df["Circ. Electoral"] + "-" + df["Local"] + "-" + df["Nro. Mesa"].fillna(0).astype(int).astype(str)\
        + "-" + df["Tipo de mesa"].fillna("NA").astype(str) + "-" + df["Mesas Fusionadas"].astype(str)

In [6]:
df.Candidato.unique()

array(['CAROLINA GOIC BOROEVIC', 'JOSE ANTONIO KAST RIST',
       'SEBASTIAN PIÑERA ECHENIQUE', 'ALEJANDRO  GUILLIER ALVAREZ',
       'BEATRIZ SANCHEZ MUÑOZ', 'MARCO  ENRIQUEZ-OMINAMI GUMUCIO',
       'EDUARDO ARTES BRICHETTI', 'ALEJANDRO NAVARRO BRAIN',
       'Votos Nulos', 'Votos Blancos', nan, 'TOTALES'], dtype=object)

In [7]:
df = df[~df["Candidato"].isin(["VOTOS NULOS", "VOTOS EN BLANCO", "Votos Blancos", "Votos Nulos", "TOTALES", np.nan])]

In [8]:
df = df.dropna(axis=1, how="all")

df["Votos TRICEL"] = df["Votos TRICEL"].fillna(0)
df["rank"] = df.groupby(["polling_id"])["Votos TRICEL"].rank("min", ascending=False).astype(int)

In [9]:
tt = df.groupby(["polling_id", "Candidato"]).agg({"Votos TRICEL": "sum"})
tt["rate"] = tt.groupby(level=[0]).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt = tt[["polling_id", "Candidato", "rate"]]

In [10]:
df = pd.merge(df, tt, on=["polling_id", "Candidato"])

In [11]:
df_filtered = df[["polling_id", "Candidato", "Votos TRICEL", "rank", "rate"]]\
    .rename(columns={"Candidato": "candidate", "Votos TRICEL": "value"})

In [12]:
df_filtered["candidate"] = df_filtered["candidate"].apply(lambda x: x.strip())

In [13]:
df_filtered.candidate.unique()

array(['CAROLINA GOIC BOROEVIC', 'JOSE ANTONIO KAST RIST',
       'SEBASTIAN PIÑERA ECHENIQUE', 'ALEJANDRO  GUILLIER ALVAREZ',
       'BEATRIZ SANCHEZ MUÑOZ', 'MARCO  ENRIQUEZ-OMINAMI GUMUCIO',
       'EDUARDO ARTES BRICHETTI', 'ALEJANDRO NAVARRO BRAIN'], dtype=object)

In [14]:
print(df_filtered.shape)
df_filtered.head()

(343120, 5)


,polling_id,candidate,value,rank,rate
0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,CAROLINA GOIC BOROEVIC,3.0,6,0.020270
1,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,JOSE ANTONIO KAST RIST,6.0,5,0.040541
2,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,SEBASTIAN PIÑERA ECHENIQUE,96.0,1,0.648649
3,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,ALEJANDRO GUILLIER ALVAREZ,19.0,2,0.128378
4,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,BEATRIZ SANCHEZ MUÑOZ,12.0,3,0.081081


In [15]:
df_filtered.to_csv(f"data_output/Chile/{year}_{election_round}.csv.gz", compression="gzip", index=False)

In [16]:
df

,Región,Provincia,Circ.Senatorial,Distrito,Comuna,Circ.Electoral,Local,Mesa,Tipo mesa,Mesas Fusionadas,Electores,Nro.voto,Candidato,Votos TRICEL,polling_id,rank,rate
0,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,1.0,CAROLINA GOIC BOROEVIC,3.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,6,0.020270
1,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,2.0,JOSE ANTONIO KAST RIST,6.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,5,0.040541
2,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,3.0,SEBASTIAN PIÑERA ECHENIQUE,96.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,1,0.648649
3,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,4.0,ALEJANDRO GUILLIER ALVAREZ,19.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,2,0.128378
4,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,5.0,BEATRIZ SANCHEZ MUÑOZ,12.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,3,0.081081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343115,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,26.0,V,26V-27V,261.0,4.0,ALEJANDRO GUILLIER ALVAREZ,21.0,PUTRE-PUTRE-LICEO GRANADEROS-26-V-26V-27V,1,0.269231
343116,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,26.0,V,26V-27V,261.0,5.0,BEATRIZ SANCHEZ MUÑOZ,14.0,PUTRE-PUTRE-LICEO GRANADEROS-26-V-26V-27V,4,0.179487
343117,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,26.0,V,26V-27V,261.0,6.0,MARCO ENRIQUEZ-OMINAMI GUMUCIO,4.0,PUTRE-PUTRE-LICEO GRANADEROS-26-V-26V-27V,5,0.051282
343118,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,26.0,V,26V-27V,261.0,7.0,EDUARDO ARTES BRICHETTI,0.0,PUTRE-PUTRE-LICEO GRANADEROS-26-V-26V-27V,8,0.000000


In [17]:
list(df)

['Región',
 'Provincia',
 'Circ.Senatorial',
 'Distrito',
 'Comuna',
 'Circ.Electoral',
 'Local',
 'Mesa',
 'Tipo mesa',
 'Mesas Fusionadas',
 'Electores',
 'Nro.voto',
 'Candidato',
 'Votos TRICEL',
 'polling_id',
 'rank',
 'rate']

In [22]:
df

,Región,Provincia,Circ.Senatorial,Distrito,Comuna,Circ.Electoral,Local,Mesa,Tipo mesa,Mesas Fusionadas,Electores,Nro.voto,Candidato,Votos TRICEL,polling_id,rank,rate
0,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,1.0,CAROLINA GOIC BOROEVIC,3.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,6,0.020270
1,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,2.0,JOSE ANTONIO KAST RIST,6.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,5,0.040541
2,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,3.0,SEBASTIAN PIÑERA ECHENIQUE,96.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,1,0.648649
3,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,4.0,ALEJANDRO GUILLIER ALVAREZ,19.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,2,0.128378
4,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0,5.0,BEATRIZ SANCHEZ MUÑOZ,12.0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,3,0.081081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343115,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,26.0,V,26V-27V,261.0,4.0,ALEJANDRO GUILLIER ALVAREZ,21.0,PUTRE-PUTRE-LICEO GRANADEROS-26-V-26V-27V,1,0.269231
343116,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,26.0,V,26V-27V,261.0,5.0,BEATRIZ SANCHEZ MUÑOZ,14.0,PUTRE-PUTRE-LICEO GRANADEROS-26-V-26V-27V,4,0.179487
343117,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,26.0,V,26V-27V,261.0,6.0,MARCO ENRIQUEZ-OMINAMI GUMUCIO,4.0,PUTRE-PUTRE-LICEO GRANADEROS-26-V-26V-27V,5,0.051282
343118,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,26.0,V,26V-27V,261.0,7.0,EDUARDO ARTES BRICHETTI,0.0,PUTRE-PUTRE-LICEO GRANADEROS-26-V-26V-27V,8,0.000000


In [21]:
df[["polling_id", "Región", "Provincia", "Circ.Senatorial", "Distrito", "Comuna", "Circ.Electoral", "Local",
        "Mesa", "Tipo mesa", "Mesas Fusionadas", "Electores"]].drop_duplicates()

,polling_id,Región,Provincia,Circ.Senatorial,Distrito,Comuna,Circ.Electoral,Local,Mesa,Tipo mesa,Mesas Fusionadas,Electores
0,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-1-M-1M-5M,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,1.0,M,1M-5M,419.0
8,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-2-M-2M-3M,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,2.0,M,2M-3M,441.0
16,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-2-V-2V-3V-4V,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,2.0,V,2V-3V-4V,383.0
24,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-4-M-4M-6M-7M,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,4.0,M,4M-6M-7M,417.0
32,CAMIÑA-CAMIÑA-ESCUELA CAMIÑA-5-V-5V-6V-7V,1.0,DEL TAMARUGAL,2a Circunscripción,2° Distrito,CAMIÑA,CAMIÑA,ESCUELA CAMIÑA,5.0,V,5V-6V-7V,378.0
...,...,...,...,...,...,...,...,...,...,...,...,...
343080,PUTRE-PUTRE-LICEO GRANADEROS-15-V-15V-16V,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,15.0,V,15V-16V,376.0
343088,PUTRE-PUTRE-LICEO GRANADEROS-17-V-17V-18V-19V,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,17.0,V,17V-18V-19V,391.0
343096,PUTRE-PUTRE-LICEO GRANADEROS-20-V-20V-21V-22V,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,20.0,V,20V-21V-22V,403.0
343104,PUTRE-PUTRE-LICEO GRANADEROS-23-V-23V-24V-25V,15.0,PARINACOTA,1a Circunscripción,1er Distrito,PUTRE,PUTRE,LICEO GRANADEROS,23.0,V,23V-24V-25V,394.0


In [19]:
if year == 2021:

    df_location = df[["polling_id", "Nro. Región", "Región", "Provincia", "Circ. Senatorial", "Distrito", "Comuna", "Circ. Electoral", "Local",
        "Nro. Mesa", "Tipo de mesa", "Mesas Fusionadas", "Electores"]].drop_duplicates()

    df_location = df_location.rename(columns={
        "Nro. Región": "region_id",
        "Región": "region",
        "Provincia": "province",
        "Circ. Senatorial": "senate_district",
        "Distrito": "district",
        "Comuna": "commune",
        "Circ. Electoral": "electoral_circ",
        "Local": "polling_place",
        "Nro. Mesa": "polling_table",
        "Tipo de mesa": "polling_type",
        "Mesas Fusionadas": "mixed_polling",
        "Electores": "voters"
    })
    
    for col in ["region_id", "senate_district", "district", "polling_table", "voters"]:
        df_location[col] = df_location[col].astype(int)
    
elif year == 2017:
    df_location = df[["polling_id", "Región", "Provincia", "Circ.Senatorial", "Distrito", "Comuna", "Circ.Electoral", "Local",
        "Mesa", "Tipo mesa", "Mesas Fusionadas", "Electores"]].drop_duplicates()

    df_location = df_location.rename(columns={
        "Región": "region_id",
        "Provincia": "province",
        "Circ.Senatorial": "senate_district",
        "Distrito": "district",
        "Comuna": "commune",
        "Circ.Electoral": "electoral_circ",
        "Local": "polling_place",
        "Mesa": "polling_table",
        "Tipo mesa": "polling_type",
        "Mesas Fusionadas": "mixed_polling",
        "Electores": "voters"
    })
    
    for col in ["region_id", "polling_table", "voters"]:
        df_location[col] = df_location[col].astype(int)

df_location.to_csv(f"data_output/Chile/{year}_{election_round}_location.csv.gz", compression="gzip", index=False)